In [1]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
import litellm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
from typing import Optional, Dict, Any
import json
from google.adk.tools.tool_context import ToolContext
from neo4j_for_adk import graphdb

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Defining Model Constants for easier use 
MODEL_GPT = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT)

# Test LLM with a direct call
response = llm.llm_client.completion(
    model=llm.model,
    api_key=os.getenv("OPENAI_API_KEY"),
    messages=[{"role": "user", "content": "Are you ready?"}],
    tools=[]
)
print(json.dumps(response.dict(), indent=2))

print("\nOpenAI is ready for use.")

{
  "id": "chatcmpl-CA966kuogSPbTVwTt4gh37wQoYYY7",
  "created": 1756534286,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_46bff0e0c8",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Yes, I'm ready! How can I assist you today?",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "annotations": []
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 13,
    "prompt_tokens": 27,
    "total_tokens": 40,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0,
      "text_tokens": null
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0,
      "text_tokens": null,
      "image_tokens": null
    }
  },
  "service_tier": "default"
}

OpenAI is ready for use.

<img src="send_query_expln.png" alt="diagram of the send_query method from the graphdb module showing success or failure response to a query" width=400>  

In [3]:
# Sending a simple query to the database
neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

{'status': 'success', 'query_result': [{'message': 'Neo4j is Ready!'}]}


- An `Agent` in Google ADK orchestrates the interaction between the user, the LLM, and the available tools

- key parameters:
    * `name`: A unique identifier for this agent (e.g., "friendly_cypher_agent\_v1").  
    * `model`: Specifies which LLM to use. you'll use the `llm` variable we defined above.  
    * `description`: A summary of the agent's overall purpose. This is like public documentation that helps other agents decide when to delegate tasks to *this* agent.  
    * `instruction`: Detailed guidance given to the LLM on how this agent should behave, its persona, goals, and specifically *how and when* to utilize its assigned `tools`.  
    * `tools`: A list containing the actual Python tool functions the agent is allowed to use (e.g., `[say_hello]`).

**Best Practice:** 
- Choose descriptive `name` and `description` values. These are used internally by ADK and are vital for features like automatic delegation (covered later).

In [4]:
# Define a basic tool -- send a parameterized cypher query
def say_hello(person_name: str) -> dict:
    """Formats a welcome message to a named person. 

    Args:
        person_name (str): the name of the person saying hello

    Returns:
        dict: A dictionary containing the results of the query.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'query_result' key with an array of result rows.
              If 'error', includes an 'error_message' key.
    """
    return graphdb.send_query("RETURN 'Hello to you, ' + $person_name AS reply",
    {
        "person_name": person_name
    })
print(say_hello("Amrith"))
print(say_hello("RETURN 'injection attack avoided'"))

{'status': 'success', 'query_result': [{'reply': 'Hello to you, Amrith'}]}
{'status': 'success', 'query_result': [{'reply': "Hello to you, RETURN 'injection attack avoided'"}]}


In [5]:
def say_goodbye(dummy: str = "") -> dict:
    """Provides a simple farewell message to conclude the conversation."""
    return graphdb.send_query("RETURN 'Goodbye from Cypher!' as farewell")

print(say_goodbye())

{'status': 'success', 'query_result': [{'farewell': 'Goodbye from Cypher!'}]}


In [6]:
def add(x: float, y: float) -> dict:
    """Add two numbers."""
    return {"result": x + y}

def subtract(x: float, y: float) -> dict:
    """Subtract two numbers."""
    return {"result": x - y}

def multiply(x: float, y: float) -> dict:
    """Multiply two numbers."""
    return {"result": x * y}

def divide(x: float, y: float) -> dict:
    """Divide two numbers."""
    if y == 0:
        return {"error": "Division by zero"}
    return {"result": x / y}

In [7]:
# Define the Cypher Agent
hello_agent = Agent(
    name="hello_agent_v1",
    model=llm, # defined earlier in a variable
    description="Has friendly chats with a user.",
    instruction="""You are a helpful assistant, chatting with a user. 
                Be polite and friendly, introducing yourself and asking who the user is. 

                If the user provides their name, use the 'say_hello' tool to get a custom greeting.
                If the tool returns an error, inform the user politely. 
                If the tool is successful, present the reply.
                """,
    tools=[say_hello], # Pass the function directly
)

print(f"Agent '{hello_agent.name}' created.")

Agent 'hello_agent_v1' created.


<img src="event_loop_3.png"  alt="diagram of Agent Development Kit runtime showing the Runner component handling an Event Loop with Services for the User" width=400> 

* `SessionService`: Responsible for managing conversation history and state for different users and sessions. The `InMemorySessionService` is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged.  
* `Runner`: The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the `SessionService`, and yields events representing the progress of the interaction.

In [8]:
app_name = hello_agent.name + "_app"
user_id = hello_agent.name + "_user"
session_id = hello_agent.name + "_session_01"
    
# Initializing a session service and a session
session_service = InMemorySessionService()
await session_service.create_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id
)
    
runner = Runner(
    agent=hello_agent,
    app_name=app_name,
    session_service=session_service
)

print(f"Runner for agent '{runner.agent.name}' initialized.")
print(f"App Name: {runner.app_name}")
print(f"User ID: {runner.agent.description}")
print(f"Input Schema: {runner.agent.input_schema}")
print(f"Tools: {runner.agent.tools}")
print(f"Model: {runner.agent.model}")
print(f"Model Config: {runner.agent.model_config}")
print(f"Schema: {runner.agent.schema}")
print(f"Sub Agents: {runner.agent.sub_agents}")
print(f"Parent Agent: {runner.agent.parent_agent}")
print(f"Agent Instruction: {runner.agent.instruction}")


Runner for agent 'hello_agent_v1' initialized.
App Name: hello_agent_v1_app
User ID: Has friendly chats with a user.
Input Schema: None
Tools: [<function say_hello at 0x7761a7931e40>]
Model: model='openai/gpt-4o' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x77619fd73490>
Model Config: {'arbitrary_types_allowed': True, 'extra': 'forbid'}
Schema: <bound method BaseModel.schema of <class 'google.adk.agents.llm_agent.LlmAgent'>>
Sub Agents: []
Parent Agent: None
Agent Instruction: You are a helpful assistant, chatting with a user. 
                Be polite and friendly, introducing yourself and asking who the user is. 

                If the user provides their name, use the 'say_hello' tool to get a custom greeting.
                If the tool returns an error, inform the user politely. 
                If the tool is successful, present the reply.
                


In [9]:
user_message = "Hello, I'm Amrith"
print(f"\n>>> User Message: {user_message}")

content = types.Content(role='user', parts=[types.Part(text=user_message)])

final_response_text = "Agent did not produce a final response." # Default will be replaced if the agent produces a final response.

verbose = True
async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
    if verbose:
        print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")
    
    # Key Concept: is_final_response() marks the concluding message for the turn.
    if event.is_final_response():
        if event.content and event.content.parts:
            final_response_text = event.content.parts[0].text # Assuming text response in the first part
        elif event.actions and event.actions.escalate: # Handle potential errors/escalations
            final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
        break # Stop processing events once the final response is found

print(f"<<< Agent Response: {final_response_text}")


>>> User Message: Hello, I'm Amrith
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'person_name': 'Amrith'
    },
    id='call_cPuAXSnEmpxnkianQXwQUMEm',
    name='say_hello'
  )
)] role='model'
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_cPuAXSnEmpxnkianQXwQUMEm',
    name='say_hello',
    response={
      'query_result': [
        {
          'reply': 'Hello to you, Amrith'
        },
      ],
      'status': 'success'
    }
  )
)] role='user'
  [Event] Author: hello_agent_v1, Type: Event, Final: True, Content: parts=[Part(
  text="Hello to you, Amrith! It's great to meet you. How can I assist you today?"
)] role='model'
<<< Agent Response: Hello to you, Amrith! It's great to meet you. How can I assist you today?


In [10]:
class AgentCaller:
    """A simple wrapper class for interacting with an ADK agent."""
    
    def __init__(self, agent: Agent, runner: Runner, 
                 user_id: str, session_id: str):
        """Initialize the AgentCaller with required components."""
        self.agent = agent
        self.runner = runner
        self.user_id = user_id
        self.session_id = session_id


    def get_session(self):
        return self.runner.session_service.get_session(app_name=self.runner.app_name, user_id=self.user_id, session_id=self.session_id)

    
    async def call(self, user_message: str, verbose: bool = False):
        """Call the agent with a query and return the response."""
        print(f"\n>>> User Message: {user_message}")

        # Prepare the user's message in ADK format
        content = types.Content(role='user', parts=[types.Part(text=user_message)])

        final_response_text = "Agent did not produce a final response." 
        
        verbose = True
        
        # Key Concept: run_async executes the agent logic and yields Events.
        # We iterate through events to find the final answer.
        async for event in self.runner.run_async(user_id=self.user_id, session_id=self.session_id, new_message=content):
            # You can uncomment the line below to see *all* events during execution
            if verbose:
                print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

            # Key Concept: is_final_response() marks the concluding message for the turn.
            if event.is_final_response():
                if event.content and event.content.parts:
                    # Assuming text response in the first part
                    final_response_text = event.content.parts[0].text
                elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                    final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
                break # Stop processing events once the final response is found

        print(f"<<< Agent Response: {final_response_text}")
        return final_response_text


In [11]:
async def make_agent_caller(agent: Agent, initial_state: Optional[Dict[str, Any]] = {}) -> AgentCaller:
    """Create and return an AgentCaller instance for the given agent."""
    app_name = agent.name + "_app"
    user_id = agent.name + "_user"
    session_id = agent.name + "_session_01"
    
    # Initialize a session service and a session
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        state=initial_state
    )
    
    runner = Runner(
        agent=agent,
        app_name=app_name,
        session_service=session_service
    )
    
    return AgentCaller(agent, runner, user_id, session_id)


In [12]:
hello_agent_caller = await make_agent_caller(hello_agent)

async def run_conversation():
    await hello_agent_caller.call("Hello I'm Amrith")

    await hello_agent_caller.call("I am excited")
    
    await hello_agent_caller.call("Multiply 20 * 10")

await run_conversation()


>>> User Message: Hello I'm Amrith
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'person_name': 'Amrith'
    },
    id='call_qcnmk8soc7nPCSJGUGJoo55d',
    name='say_hello'
  )
)] role='model'
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_qcnmk8soc7nPCSJGUGJoo55d',
    name='say_hello',
    response={
      'query_result': [
        {
          'reply': 'Hello to you, Amrith'
        },
      ],
      'status': 'success'
    }
  )
)] role='user'
  [Event] Author: hello_agent_v1, Type: Event, Final: True, Content: parts=[Part(
  text='Hello to you, Amrith! How can I assist you today?'
)] role='model'
<<< Agent Response: Hello to you, Amrith! How can I assist you today?

>>> User Message: I am excited
  [Event] Author: hello_agent_v1, Type: Event, Final: True, Content: parts=[Part(
  text="That's wonderful, Amrith! Wh

In [13]:
greeting_subagent = Agent(
    model=llm,
    name="greeting_subagent_v1",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                "Use the 'say_hello' tool to generate the greeting. "
                "If the user provides their name, make sure to pass it to the tool. "
                "Do not engage in any other conversation or tasks.",
    description="Handles simple greetings and hellos using the 'say_hello' tool.",
    tools=[say_hello]
)
print(f"✅ Agent '{greeting_subagent.name}' created.")


farewell_subagent = Agent(
    model=llm,
    name="farewell_subagent_v1",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                "Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", 
    tools=[say_goodbye],
)
print(f"✅ Agent '{farewell_subagent.name}' created.")

math_subagent = Agent(
    model=llm,
    name="math_subagent_v1",
    instruction="You are the Math Agent. Your ONLY task is to perform basic arithmetic operations. "
                "Use the 'calculator' tool to perform calculations. "
                "If the user provides numbers and an operation, make sure to pass them to the tool. "
                "Do not engage in any other conversation or tasks.",
    description="Handles simple arithmetic operations using the 'calculator' tool.",
    tools=[add, subtract, multiply, divide],
)
print(f"✅ Agent '{math_subagent.name}' created.")


✅ Agent 'greeting_subagent_v1' created.
✅ Agent 'farewell_subagent_v1' created.
✅ Agent 'math_subagent_v1' created.


In [14]:
root_agent = Agent(
    name="friendly_agent_team_v1", 
    model=llm,
    description="The main coordinator agent. Delegates greetings/farewells to specialists.",
    instruction="""You are the main Agent coordinating a team. Your primary responsibility is to be friendly.
 
                You have specialized sub-agents: 
                1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 
                3. 'math_agent': Handles basic arithmetic like addition, subtraction, multiplication, division. Delegate to it for these.

                Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. 
                If it's a farewell, delegate to 'farewell_agent'. 
                If it's a math question, delegate to 'math_agent'.

                For anything else, respond appropriately or state you cannot handle it.
                """,
    tools=[],
    sub_agents=[
        greeting_subagent,
        farewell_subagent,
        math_subagent
    ]
)
print(f"✅ Root Agent '{root_agent.name}' created with sub-agents: {[agent.name for agent in root_agent.sub_agents]}.")

✅ Root Agent 'friendly_agent_team_v1' created with sub-agents: ['greeting_subagent_v1', 'farewell_subagent_v1', 'math_subagent_v1'].


In [15]:
from helper import make_agent_caller

root_agent_caller = await make_agent_caller(root_agent)

print(f"Runner for agent '{root_agent_caller.agent.name}' initialized.")
print(f"User ID: {root_agent_caller.agent.description}")
print(f"Input Schema: {root_agent_caller.agent.input_schema}")
print(f"Tools: {root_agent_caller.agent.tools}")
print(f"Model: {root_agent_caller.agent.model}")
print(f"Model Config: {root_agent_caller.agent.model_config}")
print(f"Schema: {root_agent_caller.agent.schema}")
print(f"Parent Agent: {root_agent_caller.agent.parent_agent}")
print(f"Agent Instruction: {root_agent_caller.agent.instruction}")
print("Sub Agents:")
for agent in root_agent_caller.agent.sub_agents:
    print(f"  - Name: {agent.name}")
    print(f"    Description: {agent.description}")
    print(f"    Tools: {[tool.__name__ for tool in agent.tools]}")
    print(f"    Instruction: {agent.instruction[:80]}{'...' if len(agent.instruction) > 80 else ''}")
    print()

Runner for agent 'friendly_agent_team_v1' initialized.
User ID: The main coordinator agent. Delegates greetings/farewells to specialists.
Input Schema: None
Tools: []
Model: model='openai/gpt-4o' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x77619fd73490>
Model Config: {'arbitrary_types_allowed': True, 'extra': 'forbid'}
Schema: <bound method BaseModel.schema of <class 'google.adk.agents.llm_agent.LlmAgent'>>
Parent Agent: None
Agent Instruction: You are the main Agent coordinating a team. Your primary responsibility is to be friendly.

                You have specialized sub-agents: 
                1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 
                3. 'math_agent': Handles basic arithmetic like addition, subtraction, multiplication, division. Delegate to it for these.

                Analyze the 

In [16]:
async def run_team_conversation():
    await root_agent_caller.call("Hello I'm Amrith")

    await root_agent_caller.call("What is 5 + 3 * 10?")

    await root_agent_caller.call("Thanks, bye!")

await run_team_conversation()


>>> User Query: Hello I'm Amrith
<<< Agent Response: Hello to you, Amrith!

>>> User Query: What is 5 + 3 * 10?
<<< Agent Response: The result of \(5 + 3 \times 10\) is 35.

>>> User Query: Thanks, bye!
<<< Agent Response: Goodbye from Cypher!


In [17]:
def say_hello_stateful(user_name:str, tool_context:ToolContext):
    """Says hello to the user, recording their name into state.
    
    Args:
        user_name (str): The name of the user.
    """
    tool_context.state["user_name"] = user_name
    print("\ntool_context.state['user_name']:", tool_context.state["user_name"])
    return graphdb.send_query(
        f"RETURN 'Hello to you, ' + $user_name + '.' AS reply",
    {
        "user_name": user_name
    })

In [18]:
def say_goodbye_stateful(tool_context: ToolContext, dummy: str) -> dict:
    """Says goodbye to the user, reading their name from state."""
    user_name = tool_context.state.get("user_name", "stranger")
    print("\ntool_context.state['user_name']:", user_name)
    return graphdb.send_query("RETURN 'Goodbye, ' + $user_name + ', nice to chat with you!' AS reply",
    {
        "user_name": user_name
    })

In [19]:
def add_calc_stateful(a: int, b: int, tool_context: ToolContext) -> dict:
    """Adds two numbers and stores the result in state."""
    result = a + b
    tool_context.state["calc_result"] = result
    print("\ntool_context.state['calc_result']:", tool_context.state["calc_result"])
    return graphdb.send_query(
        f"RETURN 'The sum of ' + $a + ' and ' + $b + ' is ' + $result AS reply",
        {
            "a": a,
            "b": b,
            "result": result
        }
    )

def subtract_calc_stateful(a: int, b: int, tool_context: ToolContext) -> dict:
    """Subtracts two numbers and stores the result in state."""
    result = a - b
    tool_context.state["calc_result"] = result
    print("\ntool_context.state['calc_result']:", tool_context.state["calc_result"])
    return graphdb.send_query(
        f"RETURN 'The difference of ' + $a + ' and ' + $b + ' is ' + $result AS reply",
        {
            "a": a,
            "b": b,
            "result": result
        }
    )
    
def multiply_calc_stateful(a: int, b: int, tool_context: ToolContext) -> dict:
    """Multiplies two numbers and stores the result in state."""
    result = a * b
    tool_context.state["calc_result"] = result
    print("\ntool_context.state['calc_result']:", tool_context.state["calc_result"])
    return graphdb.send_query(
        f"RETURN 'The product of ' + $a + ' and ' + $b + ' is ' + $result AS reply",
        {
            "a": a,
            "b": b,
            "result": result
        }
    )

def divide_calc_stateful(a: int, b: int, tool_context: ToolContext) -> dict:
    """Divides two numbers and stores the result in state."""
    if b == 0:
        return {"reply": "Division by zero error."}
    result = a / b
    tool_context.state["calc_result"] = result
    print("\ntool_context.state['calc_result']:", tool_context.state["calc_result"])
    return graphdb.send_query(
        f"RETURN 'The division of ' + $a + ' by ' + $b + ' is ' + $result AS reply",
        {
            "a": a,
            "b": b,
            "result": result
        }
    )


In [20]:
print("✅ State-aware 'say_hello_stateful' and 'say_goodbye_stateful' tools defined, along with basic math tools.")

✅ State-aware 'say_hello_stateful' and 'say_goodbye_stateful' tools defined, along with basic math tools.


In [21]:
# define a stateful greeting agent. the only difference is that this agent will use the stateful say_hello_stateful tool
greeting_agent_stateful = Agent(
    model=llm,
    name="greeting_agent_stateful_v1",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
    description="Handles simple greetings and hellos using the 'say_hello_stateful' tool.",
    tools=[say_hello_stateful],
)
print(f"✅ Agent '{greeting_agent_stateful.name}' redefined.")

farewell_agent_stateful = Agent(
    model=llm,
    name="farewell_agent_stateful_v1",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye_stateful' tool. Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye_stateful' tool.",
    tools=[say_goodbye_stateful],
)
print(f"✅ Agent '{farewell_agent_stateful.name}' redefined.")

math_agent_stateful = Agent(
    model=llm,
    name="math_agent_stateful_v1",
    instruction="You are the Math Agent. Your ONLY task is to perform basic arithmetic operations using the stateful math tools. Do nothing else.",
    description="Handles basic math operations using stateful tools.",
    tools=[
        add_calc_stateful,
        subtract_calc_stateful,
        multiply_calc_stateful,
        divide_calc_stateful
    ],
)
print(f"✅ Agent '{math_agent_stateful.name}' redefined.")


✅ Agent 'greeting_agent_stateful_v1' redefined.
✅ Agent 'farewell_agent_stateful_v1' redefined.
✅ Agent 'math_agent_stateful_v1' redefined.


In [22]:
root_agent_stateful = Agent(
    name="friendly_team_stateful",
    model=llm,
    description="The main coordinator agent. Delegates greetings/farewells to specialists.",
    instruction="""You are the main Agent coordinating a team. Your primary responsibility is to be friendly.

                You have specialized sub-agents: 
                1. 'greeting_agent_stateful': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent_stateful': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 
                3. 'math_agent_stateful' : Handles basic arithmetic like addition, subtraction, multiplication, division. Delegate to it for these.

                Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. 
                If it's a farewell, delegate to 'farewell_agent'. 
                If it's a math question, delegate to 'math_agent'.

                For anything else, respond appropriately or state you cannot handle it.
                """,
        tools=[],
        sub_agents=[greeting_agent_stateful, farewell_agent_stateful, math_agent_stateful], 
    )

print(f"✅ Root Agent '{root_agent_stateful.name}' created using agents with stateful tools.")


✅ Root Agent 'friendly_team_stateful' created using agents with stateful tools.


In [23]:
from helper import make_agent_caller

root_stateful_caller = await make_agent_caller(root_agent_stateful)

session = await root_stateful_caller.get_session()
print(f"Initial State: {session.state}")

print(f"Runner for agent '{root_stateful_caller.agent.name}' initialized.")
print(f"User ID: {root_stateful_caller.agent.description}")
print(f"Input Schema: {root_stateful_caller.agent.input_schema}")
print(f"Tools: {root_stateful_caller.agent.tools}")
print(f"Model: {root_stateful_caller.agent.model}")
print(f"Model Config: {root_stateful_caller.agent.model_config}")
print(f"Schema: {root_stateful_caller.agent.schema}")
print(f"Parent Agent: {root_stateful_caller.agent.parent_agent}")
print(f"Agent Instruction: {root_stateful_caller.agent.instruction}")
print("Sub Agents:")
for agent in root_stateful_caller.agent.sub_agents:
    print(f"  - Name: {agent.name}")
    print(f"    Description: {agent.description}")
    print(f"    Tools: {[tool.__name__ for tool in agent.tools]}")
    print(f"    Instruction: {agent.instruction[:80]}{'...' if len(agent.instruction) > 80 else ''}")
    print()

Initial State: {}
Runner for agent 'friendly_team_stateful' initialized.
User ID: The main coordinator agent. Delegates greetings/farewells to specialists.
Input Schema: None
Tools: []
Model: model='openai/gpt-4o' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x77619fd73490>
Model Config: {'arbitrary_types_allowed': True, 'extra': 'forbid'}
Schema: <bound method BaseModel.schema of <class 'google.adk.agents.llm_agent.LlmAgent'>>
Parent Agent: None
Agent Instruction: You are the main Agent coordinating a team. Your primary responsibility is to be friendly.

                You have specialized sub-agents: 
                1. 'greeting_agent_stateful': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent_stateful': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 
                3. 'math_agent_stateful' : Handles basic arithmetic like addition, subtraction, multiplication, division. Delegate 

In [24]:
async def run_stateful_conversation():
    await root_stateful_caller.call("Hello, I'm ABK!")
    await root_stateful_caller.call("What is 5 + 3?")
    await root_stateful_caller.call("Thanks, bye!")

await run_stateful_conversation()

session = await root_stateful_caller.get_session()

print(f"\nFinal State: {session.state}")


>>> User Query: Hello, I'm ABK!

tool_context.state['user_name']: ABK
<<< Agent Response: Hello to you, ABK.

>>> User Query: What is 5 + 3?

tool_context.state['calc_result']: 8
<<< Agent Response: The sum of 5 and 3 is 8.

>>> User Query: Thanks, bye!

tool_context.state['user_name']: ABK
<<< Agent Response: Goodbye, ABK, nice to chat with you!

Final State: {'user_name': 'ABK', 'calc_result': 8}


In [25]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await root_stateful_caller.call(user_query)
        print(f"Response: {response}")

await run_interactive_conversation()


>>> User Query: hello

tool_context.state['user_name']: ABK
<<< Agent Response: Hello to you, ABK.
Response: Hello to you, ABK.

>>> User Query: How are you?
<<< Agent Response: I'm just a virtual presence, but I'm here to help you with anything you need. How can I assist you today?
Response: I'm just a virtual presence, but I'm here to help you with anything you need. How can I assist you today?

>>> User Query: help me solve (4+12-8*4)^2

tool_context.state['calc_result']: 16

tool_context.state['calc_result']: 32

tool_context.state['calc_result']: -16

tool_context.state['calc_result']: 256
<<< Agent Response: The result of \((4 + 12 - 8 \times 4)^2\) is 256.
Response: The result of \((4 + 12 - 8 \times 4)^2\) is 256.

>>> User Query: byee AI

tool_context.state['user_name']: ABK
<<< Agent Response: Goodbye, ABK, nice to chat with you!
Response: Goodbye, ABK, nice to chat with you!

>>> User Query: quit

tool_context.state['user_name']: ABK
<<< Agent Response: Goodbye, ABK, nice t